# Predicting 30-day readmission for diabetic patients

By Rebecca Barter

Data from: https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008

# This presentation isn't about teaching code

Try not to focus on the code itself too much. You're not expected to understand it all!

It's useful to see examples of sophisticated code. To learn about the coding style I use in this presentation, the following blog posts will be helpeful

- **tidyverse** (ggplot, %>%, mutate, ...): http://www.rebeccabarter.com/blog/2019-08-05_base_r_to_tidyverse/

- **purrr** (map functions): http://www.rebeccabarter.com/blog/2019-08-19_purrr/

- **tidymodels** (ML): http://www.rebeccabarter.com/blog/2020-03-25_machine_learning/

## Loading packages and data


In [ ]:
library(tidyverse)
library(tidymodels)
# load the data
diabetic_orig <- read_csv("../data/raw_data/diabetic_data.csv")

## First glimpse of the data

Identify the dimension of the data:

In [ ]:
# identify the dimension of the data
print(dim(diabetic_orig))

Print out the first 6 rows of the data

In [ ]:
# set width = Inf so the tibble does not suppress columns in the console
head(diabetic_orig) %>%
  print(width = Inf)

Things to notice:


- The variables are mostly categorical and coded using characters such as "No", "Steady", "Up", etc

- `age` is coded categorically as intervals.

- Missing values seem to be coded as `?`.

- `admission_type_id` is coded numerically.

- The diagnosis variables `diag_1`, `diag_1`, and `diag_1` are coded using ICD diagnosis codes.


## Missing value check

In [ ]:
diabetic_orig %>%
  # apply a function to every column
  map_dbl(function(.var) {
      sum(.var == "?") / length(.var)
    }) %>%
  # arrange in decreasing order
  sort(decreasing = TRUE) %>%
  print()

## Unique values check

In [ ]:
diabetic_orig %>%
  # apply n_distinct() to every column
  map_dbl(n_distinct) %>%
  sort() %>%
  print()

## Check ranges

In [ ]:
diabetic_orig %>% 
  # remove the class and id variables
  select_if(is.numeric) %>%
  select(-encounter_id, -patient_nbr) %>%
  # calculate the max and min for each column and put the results in a df
  map_df(~data.frame(min = min(., na.rm = T), 
                     max = max(., na.rm = T)), .id = "variable") %>%
  print()

## Clean the data

I wrote an R script (`clean.R`) that cleans the data, then I saved the clean data as "diabetic_data_clean.csv" in a new folder.

In [ ]:
diabetic_clean <- read_csv("../data/processed_data/diabetic_data_clean.csv")

In [ ]:
head(diabetic_clean) %>%
  print(width = Inf)

## Explore the clean data

### How many unique encounters for each patient?

In [ ]:
diabetic_clean %>%
  # count the number of times each patient number occurs in the data
  count(patient_nbr) %>%
  arrange(desc(n)) %>%
  print()

Visualize it!

In [ ]:
options(repr.plot.width=16, repr.plot.height=6) # set size of plot below

# count the number of times each patient number occurs in the data
diabetic_clean %>%
  count(patient_nbr) %>%
  arrange(desc(n)) %>%
  # make a histogram of these counts
  ggplot() +
  geom_histogram(aes(x = n), binwidth = 1, col = "white") +
  scale_x_continuous("Number of encounters", breaks = 1:40) +
  scale_y_continuous("Number of patients", expand = c(0, 0)) +
  theme_classic(base_size = 20) 

## Explore the clean data

### How many encounters result in readmission in 30 days?

In [ ]:
diabetic_clean %>% 
  count(readmitted) %>%
  print()

## Explore the clean data

### How is each variable related to `readmission`?

In [ ]:
options(repr.plot.width=10, repr.plot.height=6) # set size of plot below

diabetic_clean %>%
  ggplot() +
  geom_boxplot(aes(x = as.factor(readmitted), y = time_in_hospital)) +
  theme_classic(base_size = 20) 

I want to plot this for each variable -- write a function!

In [ ]:
# a function to print the boxplots for any variable
plotBoxplots <- function(.var) {
  diabetic_clean %>%
    ggplot() +
    geom_boxplot(aes(x = readmitted, y = .data[[.var]])) +
    theme_classic(base_size = 20) +
    ggtitle(.var) +
    scale_x_discrete(NULL, labels = c("Not readmitted", "readmitted"))
}

In [ ]:
options(repr.plot.width=10, repr.plot.height=6) # set size of plot below

plotBoxplots("age")

Then make a grid of plots

In [ ]:
boxplot_list <- diabetic_clean %>%
  # remove the variables we don't want to plot
  select_if(is.numeric) %>%
  select(-patient_nbr, -encounter_id) %>%
  colnames %>%
  # apply the plotBoxplots function to each column name
  map(plotBoxplots)

In [ ]:
options(repr.plot.width=18, repr.plot.height=18) # set size of plot below

gridExtra::grid.arrange(grobs = boxplot_list)

## Explore the clean data

### How are **categorical variables** related to `readmission`?

In [ ]:
printDotPlot <- function(.var, legend = TRUE, .base_size = 20) {
  proportions <- diabetic_clean %>%
    count(readmitted, .data[[.var]]) %>%
    drop_na() %>%
    group_by(readmitted) %>%
    mutate(prop = round(n / sum(n), 3)) %>%
    ungroup() 
  p <- proportions %>%
    ggplot() +
    geom_point(aes(x = prop, y = .data[[.var]], col = readmitted),
               size = 6, alpha = 0.8) +
    theme_classic(base_size = .base_size) +
    theme(panel.grid.major.y = element_line(color = "grey50"),
          axis.line = element_blank()) 
    scale_x_continuous("Proportion of patients", limits = c(0, 1))
  if (legend) {
      p + theme(legend.position = "top")
  } else {
      p + theme(legend.position = "none")
  }
}

In [ ]:
options(repr.plot.width=10, repr.plot.height=6) # set size of plot below

printDotPlot("insulin")

In [ ]:
options(repr.plot.width=18, repr.plot.height=26) # set size of plot below

dot_plot_list <- diabetic_clean %>% 
  select_if(is.character) %>%
  colnames %>%
  map(~printDotPlot(., legend = FALSE, .base_size = 14))
gridExtra::grid.arrange(grobs = dot_plot_list, ncol = 4)

## Pre-process data for modeling using tidymodels

### Split the data into training and testing

In [ ]:
# remove ID variables
diabetic_clean <- diabetic_clean %>%
  select(-encounter_id, -patient_nbr)

# create a split object
diabetic_split <- initial_split(diabetic_clean)

In [ ]:
diabetic_split %>% print

If you want to extract the training and testing datasets you can

In [ ]:
diabetic_train <- training(diabetic_split)
diabetic_test <- testing(diabetic_split)

In [ ]:
dim(diabetic_train)

In [ ]:
head(diabetic_train) %>%
  print()

## Pre-process using recipes

In [ ]:
diabetic_recipe <-
  # which consists of the formula (outcome ~ predictors)
  recipe(readmitted ~ .,
         data = diabetic_train) %>%
  # impute the missing values
  step_meanimpute(all_numeric()) %>%
  step_modeimpute(all_nominal()) %>%
  # remove features that have almost entirely identical values across all rows
  step_nzv(all_predictors()) %>%
  # convert categorical variables to dummy variables
  step_dummy(all_nominal(), -readmitted) 

If you want to extract the pre-processed data, you can

In [ ]:
diabetic_preprocessed <- prep(diabetic_recipe, diabetic_train) %>% 
  bake(diabetic_train)

In [ ]:
head(diabetic_preprocessed) %>% print(width = Inf)

## Prepare logistic regression models

In [ ]:
logistic_regression_model <-
  # specify that the model is logistic regression
  logistic_reg() %>%
  # select the engine/package that underlies the model
  set_engine("glm") %>%
  # choose either the continuous regression or binary classification mode
  set_mode("classification") 

## Combine the pre-processing recipe and the model into a workflow

In [ ]:
# LR workflow
logistic_regression_workflow <- workflow() %>%
  # add the recipe
  add_recipe(diabetic_recipe) %>%
  # add the model
  add_model(logistic_regression_model)

## Fit the models on the training set and evaluate on the test set

In [ ]:
logistic_regression_fit <- logistic_regression_workflow %>%
  # fit on the training set and evaluate on test set
  last_fit(diabetic_split)

In [ ]:
# extract predictions from the fitted model
lr_predictions <- collect_predictions(logistic_regression_fit)
# specify which metrics to use
calculate_metrics <- metric_set(roc_auc, accuracy, sens, spec)
# calculate the metrics
calculate_metrics(lr_predictions, truth = readmitted, estimate = .pred_class, .pred_readmitted) %>%
  print()

In [ ]:
# confusion matrix
table(truth = diabetic_test$readmitted, 
      estimate = lr_predictions$.pred_class)

## Define a new reprocessing recipe with downsampling

In [ ]:
diabetic_downsample_recipe <-
  # which consists of the formula (outcome ~ predictors)
  recipe(readmitted ~ .,
         data = diabetic_clean) %>%
  # impute the missing values
  step_meanimpute(all_numeric()) %>%
  step_modeimpute(all_nominal()) %>%

  #--------- add a downsampling step ----------------------
  step_downsample(readmitted) %>%

  # remove features that have almost entirely identical values across all rows
  step_nzv(all_predictors()) %>%
  # convert categorical variables to dummy variables
  step_dummy(all_nominal(), -readmitted) 

Then update the workflow

In [ ]:
# LR workflow
logistic_regression_workflow <- logistic_regression_workflow %>%
  update_recipe(diabetic_downsample_recipe)

## Re-evaluate the models

In [ ]:
logistic_regression_fit <- logistic_regression_workflow %>%
  # fit on the training set and evaluate on test set
  last_fit(diabetic_split)

In [ ]:
# extract predictions from the fitted model
lr_predictions <- collect_predictions(logistic_regression_fit)
# specify which metrics to use
calculate_metrics <- metric_set(roc_auc, accuracy, sens, spec)
# calculate the metrics
calculate_metrics(lr_predictions, truth = readmitted, estimate = .pred_class, .pred_readmitted) %>%
  print()

In [ ]:
# confusion matrix
table(truth = diabetic_test$readmitted, 
      estimate = lr_predictions$.pred_class)

## Create ROC curves on the test set

In [ ]:
options(repr.plot.width=8, repr.plot.height=8) # set size of plot below

# get ROC curve for logistic regression
logistic_regression_fit$.predictions[[1]] %>%
  roc_curve(readmitted, .pred_readmitted) %>%
  ggplot() +
  geom_line(aes(x = 1 - specificity, y = sensitivity)) +
  geom_abline(intercept = 0, slope = 1, 
              col = "grey40", linetype = "dashed") +
  scale_x_continuous(expand = c(0, 0)) +
  scale_y_continuous(expand = c(0, 0)) +
  theme_classic(base_size = 20) +
  coord_fixed()

## Examine variable importance and model coefficients

For logistic regression:

In [ ]:
# fit model on entire dataset
lr_final <- fit(logistic_regression_workflow, diabetic_clean)
# extract the logistic regression model object
lr_obj <- pull_workflow_fit(lr_final)$fit

In [ ]:
summary(lr_obj)